In [2]:
# load packages
import numpy as np
import pandas as pd
# import napari
import tifffile as tf
from skimage import io # import tiff file as ndarray
import os
import matplotlib.pyplot as plt

# import custum colormaps
from colormaps import *

# import custum functions
from STP_plotting import *
from STP_processing import *

# auto refreshes imported functions if source code changes:
%load_ext autoreload
%autoreload 2



In [8]:
# Acadia
in_path = "/mnt/labNAS/Emily/STP_for_MAPseq/4_python_output/input_tifs/norm_subtracted/STeg_220429_aligned/"
out_path = "/mnt/labNAS/Emily/STP_for_MAPseq/4_python_output/output_figs/"

metadata = pd.read_csv("stp_metadata.csv")

# get file paths
dir_list = os.listdir(in_path)

metadata['path'] = dir_list

# area list from make_masks.py
areas_masked = ["grey", "CTX", "OMCc", "aud","TH", "STR", "CP", "AMY", "P", "PG", "MB", "PAG", "SCm", 
         "SNr", "HY", "CNU", "TEa", "ECT", "VISC", "AI", "GU", "BS"]


In [5]:
# load masks
steg_masks = []
for i in range(len(areas)):
    with open("/mnt/labNAS/Emily/STP_for_MAPseq/4_python_output/input_tifs/masks/STeg_220429_"+areas[i]+".npy", "rb") as f:
            steg_masks.append(np.load(f))

mmus_masks = []
for i in range(len(areas)):
    with open("/mnt/labNAS/Emily/STP_for_MAPseq/4_python_output/input_tifs/masks/MMus_220303_"+areas[i]+".npy", "rb") as f:
            mmus_masks.append(np.load(f))


# import images
images = []
for i in range(len(dir_list)):
    images.append(tf.imread(in_path+dir_list[i]))

# import atlases
st_atlas_path = "/mnt/labNAS/Emily/STP_for_MAPseq/4_python_output/input_tifs/Steg_220429_registered_atlas_RESIZED.tif"
st_atlas = tf.imread(st_atlas_path)

mm_atlas_path = "/mnt/labNAS/Emily/STP_for_MAPseq/4_python_output/input_tifs/MMus_220303_registered_atlas_RESIZED.tif"
mm_atlas = tf.imread(mm_atlas_path)


# quantify integrated fluorescence




In [ ]:
# areas that match w/ mapseq data
areas_plot = ["OMCc", "CP", "aud", "AMY", "TH", "HY", "SNr", "SCm", "PG", "PAG", "BS"]

masks_plot = [areas_masked[areas_masked.index(area)] for area in areas_plot]

